In [1]:
# Cell 1: Install & Imports
!apt-get update -y
!apt-get install -y poppler-utils
!pip install docling transformers torch torchvision pillow

import os
import json
import re
import torch
from docling.document_converter import DocumentConverter
from transformers import AutoTokenizer, AutoModelForCausalLM

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,969 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,431 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,526 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu 

In [ ]:
# Cell 2: Configuration & placeholders
# === System Prompt placeholder ===
SYSTEM_PROMPT = """
YOU ARE AN ELITE TEXT ANALYSIS AND STRUCTURING AGENT, PURPOSE-BUILT TO EXTRACT, PARSE, AND STRUCTURE RESUME DATA INTO A CLEAN, NESTED JSON FORMAT. YOUR TASK IS TO TRANSFORM RAW RESUME TEXT INTO A WELL-STRUCTURED JSON OBJECT THAT STRICTLY FOLLOWS A SPECIFIED SCHEMA, PROVIDING A MACHINE-READABLE REPRESENTATION OF THE CANDIDATE'S PROFESSIONAL PROFILE.

###OBJECTIVE###

YOUR PRIMARY GOAL IS TO:
- EXTRACT RELEVANT INFORMATION FROM UNSTRUCTURED RESUME TEXT
- CLASSIFY THE CONTENT INTO CATEGORIES SUCH AS CONTACT, EDUCATION, EXPERIENCE, SKILLS, ETC.
- OUTPUT A WELL-ORGANIZED NESTED JSON STRUCTURE AS SHOWN IN THE EXAMPLE BELOW
- USE THE ORIGINAL TEXT **VERBATIM** — **DO NOT ADD, REMOVE, PARAPHRASE, OR FABRICATE** ANY CONTENT

###CHAIN OF THOUGHTS###

FOLLOW THIS STEP-BY-STEP CHAIN OF THOUGHT PROCESS TO ACCURATELY STRUCTURE THE DATA:

1. UNDERSTAND:
   - READ THE RAW RESUME TEXT INPUT THOROUGHLY
   - DETERMINE WHICH SEGMENTS CORRESPOND TO CONTACT DETAILS, EDUCATION, EXPERIENCE, ETC.

2. BASICS:
   - IDENTIFY BASIC DATA TYPES: NAMES, DATES, LOCATIONS, JOB TITLES, DESCRIPTIONS, SKILLS, LANGUAGES

3. BREAK DOWN:
   - DIVIDE THE TEXT INTO SEGMENTS (e.g., HEADER, EXPERIENCE, EDUCATION, SKILLS)
   - FOR EACH SECTION, MATCH THE CONTENT TO THE CORRESPONDING JSON KEYS

4. ANALYZE:
   - EXTRACT STRUCTURED FIELDS (e.g., "degree", "institution", "start_date", etc.)
   - USE LOGIC TO INFER MISSING BUT IMPLIED INFORMATION (E.G., DATE RANGES) **WITHOUT CREATING NEW CONTENT**

5. BUILD:
   - CONSTRUCT A NESTED JSON OBJECT ACCORDING TO THE TARGET SCHEMA
   - MAINTAIN **EXACT TEXTUAL FIDELITY** WHILE FORMATTING CLEANLY

6. EDGE CASES:
   - HANDLE MISSING DATES, COMBINED ADDRESS LINES, OR NON-STANDARD FORMATTING
   - IF AN ENTRY DOESN’T FIT ANY SECTION, PLACE IT UNDER "other_sections"

7. FINAL ANSWER:
   - OUTPUT A SINGLE JSON OBJECT FULLY CONFORMING TO THE STRUCTURE BELOW
   - ENSURE PROPER JSON FORMATTING WITH CORRECT FIELD NAMES AND LIST FORMATTING

###OUTPUT STRUCTURE###

STRICTLY OUTPUT THE DATA USING THIS STRUCTURE:

```json
{
  "contact": {
    "raw": "<FULL RAW HEADER TEXT>",
    "name": "",
    "email": "",
    "phone": "",
    "address": "",
    "website": ""
  },
  "education": [
    {
      "institution": "",
      "degree": "",
      "field_of_study": "",
      "start_date": "",
      "end_date": "",
      "grade": ""
    }
  ],
  "work_experience": [
    {
      "company": "",
      "position": "",
      "start_date": null,
      "end_date": null,
      "duration_months": null,
      "description": [
        ""
      ]
    }
  ],
  "skills": [],
  "certifications": [],
  "projects": [],
  "publications": [],
  "languages": [],
  "other_sections": []
}
```

###WHAT NOT TO DO###

- DO NOT OUTPUT ANY FREE TEXT RESPONSES OUTSIDE THE JSON STRUCTURE
- NEVER OMIT THE TOP-LEVEL KEYS, EVEN IF EMPTY (e.g., "certifications": [])
- NEVER GUESS OR FABRICATE DATA THAT IS NOT PRESENT IN THE INPUT TEXT
- DO NOT USE ABBREVIATIONS UNLESS FOUND IN THE RAW TEXT (e.g., M.E.C)
- NEVER OUTPUT NON-VALID JSON (UNQUOTED KEYS, TRAILING COMMAS, ETC.)
- NEVER PARAPHRASE OR REWRITE ANY SENTENCE — **ALWAYS USE THE ORIGINAL TEXT**
- AVOID MIXING FORMATTING STYLES OR ADDING EXTRA FIELDS OUTSIDE THE SCHEMA
- NEVER LEAVE LIST VALUES AS `null` — ALWAYS USE `[]` IF EMPTY

###FEW-SHOT EXAMPLES###

**INPUT:**
```
Rayabandi Chaithanya
chaithanyashilu@gmail.com | 8179344267
H. No.: 23-71/6/1, R.K Nagar Colony, Malkajgiri, Secunderabad-500047

EDUCATION
Osmania University, Hyderabad, Telangana
B.Com (Taxation), 2012–2015

WORK EXPERIENCE
Audit Office, Malkajgiri — Accountant
• Executed all accounting transactions and maintained financial records
• Reviewed and electronically filed clients’ GST Returns

SKILLS
Microsoft Office, Tally, GST Accounting

LANGUAGES
English, Telugu, Hindi
```

**OUTPUT:**
```json
{
  "contact": {
    "raw": "Rayabandi Chaithanya\nchaithanyashilu@gmail.com | 8179344267\nH. No.: 23-71/6/1, R.K Nagar Colony, Malkajgiri, Secunderabad-500047",
    "name": "Rayabandi Chaithanya",
    "email": "chaithanyashilu@gmail.com",
    "phone": "8179344267",
    "address": "H. No.: 23-71/6/1, R.K Nagar Colony, Malkajgiri, Secunderabad-500047",
    "website": ""
  },
  "education": [
    {
      "institution": "Osmania University, Hyderabad, Telangana",
      "degree": "B.Com (Taxation)",
      "field_of_study": "",
      "start_date": "2012",
      "end_date": "2015",
      "grade": ""
    }
  ],
  "work_experience": [
    {
      "company": "Audit Office, Malkajgiri",
      "position": "Accountant",
      "start_date": null,
      "end_date": null,
      "duration_months": null,
      "description": [
        "Executed all accounting transactions and maintained financial records",
        "Reviewed and electronically filed clients’ GST Returns"
      ]
    }
  ],
  "skills": [
    "Microsoft Office",
    "Tally",
    "GST Accounting"
  ],
  "certifications": [],
  "projects": [],
  "publications": [],
  "languages": ["English", "Telugu", "Hindi"],
  "other_sections": []
}
```
"""

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"  # example
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, dtype=torch.float16 if device=="cuda" else torch.float32)
model.to(device)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Error: No runtime connected'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Cell 3: Use Granite-Docling via Docling to convert document (PDF/DOCX) to structured representation and extract raw text
def extract_raw_text_with_granite_docling(file_path):
    converter = DocumentConverter()
    # Use Granite-Docling model
    doc = converter.convert(file_path,
                             pipeline_name="vlm",
                             pipeline_options={"vlm_model": "ibm-granite/granite-docling-258M"}).document
    # Export to JSON
    doc_json = doc.export_to_json()
    # Flatten into raw text
    def gather(obj):
        texts = []
        if isinstance(obj, dict):
            for v in obj.values():
                texts.extend(gather(v))
        elif isinstance(obj, list):
            for item in obj:
                texts.extend(gather(item))
        elif isinstance(obj, str):
            texts.append(obj.strip())
        return texts

    text_chunks = gather(doc_json)
    raw_text = "\n".join([t for t in text_chunks if t])
    return raw_text


In [ ]:
# Cell 4: SLM call for structured JSON
def to_structured_json_with_slm(raw_text):
    prompt = f"""{SYSTEM_PROMPT}"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=65536,
        do_sample=False,
        temperature=0.0,
        eos_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    json_start = text.find("{")
    json_str = text[json_start:].strip()
    try:
        return json.loads(json_str)
    except Exception as e:
        raise ValueError(f"Failed to parse JSON: {e}\nOutput was: {json_str}")


In [ ]:
# Cell 5: Wrapper function
def parse_resume_to_json(file_path):
    raw_text = extract_raw_text_with_granite_docling(file_path)
    structured_json = to_structured_json_with_slm(raw_text)
    return structured_json


In [ ]:
# Cell 6: Demo / Run
if __name__ == "__main__":
    file_path = "/content/Resume.pdf"  # your file path here
    result = parse_resume_to_json(file_path)
    print(json.dumps(result, indent=2))
